# BlackJack

File also uploaded at my [github - yashashwi-s](https://github.com/yashashwi-s/RLbook)

## Introduction to Blackjack

Blackjack, also known as 21, is a popular card game played in casinos. The objective of the game is to have a hand value that is closer to 21 than the dealer's hand without exceeding 21. 

### Basic Rules
- **Card Values**: 
  - Number cards (2-10) are worth their face value.
  - Face cards (King, Queen, Jack) are worth 10 points.
  - Aces can be worth 1 or 11 points, whichever is more favorable for the player.

### Gameplay
1. **Initial Deal**: 
   - Both the player and the dealer are dealt two cards. 
   - The player’s cards are usually dealt face-up, while the dealer has one card face-up and one face-down.

2. **Player's Turn**: 
   - The player has several actions they can take:
     - **Stick**: End their turn and keep their current hand.
     - **Hit**: Draw another card from the deck.
     - **Double Down**: Double their bet, take exactly one more card, and then stick.
     - **Surrender**: Forfeit the round and lose half of their bet (only allowed as the first action).

3. **Dealer's Turn**: 
   - The dealer reveals their face-down card.
   - The dealer must draw cards until their hand value is at least 17.

4. **Winning Conditions**:
   - The player wins if their hand is closer to 21 than the dealer's hand or if the dealer busts (exceeds 21).
   - The player loses if they bust or if the dealer’s hand is closer to 21.
   - If both the player and the dealer have the same hand value, it is a tie (push).

### Our Implementation
We will simulate this game in Python using OpenAI's Gym environment. The actions in our implementation are:
- **Stick (0)**
- **Hit (1)**
- **Double Down (2)**
- **Surrender (3)**


---

### Importing Libraries

In [1]:
import gym
from gym import spaces
from gym.utils import seeding
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import pandas as pd

---
---

## Helper Functions:

### Draw Cards:
 Here we assume an infinite deck, so we can have one card any number of times

In [2]:
def drawCard(np_random):
    card = np_random.choice([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10])
    return card

---

### Draw Hands:
Getting the intial hand from randomised deck

In [3]:
def drawHand(np_random):
    return [drawCard(np_random), drawCard(np_random)]

---

### Ace:
This method check for presence of a usable ace, i.e., was their ace present in the initial withdrawal of hand

In [4]:
def Ace(hand):
    return 1 in hand and sum(hand) + 10 <= 21

---

### Hand Sum:
This returns the the sum of all cards in the hand.

In [5]:
def handSum(hand):
    if Ace(hand):
        return sum(hand) + 10
    return sum(hand)

---

### Bust:
This function checks if the hand has gone bust, i.e., it  checks whether the sum of hand is more than 21

In [6]:
def bust(hand):
    return sum(hand) > 21

---

### Natural:
To check whether the initial hand has hit 21, which would result in a +1.5 reward

In [7]:
def natural(hand):
    return sorted(hand) == [1, 10]

---

### Possible:
This checks whether we have the option to double down or surrender

In [8]:
def possible(hand, actions_taken):
    return len(hand) == 2 and actions_taken == 0

---

### Compare Hands:
To compare whose hand is better than the other person

In [9]:
def compareHands(a, b):
    return float(a > b) - float(a < b)

---

### Score:

In [10]:
def score(hand):
    return 0 if bust(hand) else handSum(hand)

---
---

## Environment:

In [11]:
class BlackJackEnv(gym.Env):
    def __init__(self, natural=False):
        self.action_space = spaces.Discrete(4)
        self.observation_space = spaces.Tuple((
            spaces.Discrete(32), # Player's hand sum
            spaces.Discrete(11), # Dealer's face-up card
            spaces.Discrete(2),  # Usable ace
            spaces.Discrete(2)   # Double down or surrender possibility
        ))
        self.seed()
        self.natural = natural
        self.reset()

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def reset(self):
        self.actions_taken = 0
        self.dealer_hand = drawHand(self.np_random)
        self.player_hand = drawHand(self.np_random)
        return self._get_obs()

    def _get_obs(self):
        return (handSum(self.player_hand), self.dealer_hand[0], 
                Ace(self.player_hand), possible(self.player_hand, self.actions_taken))

    def step(self, action):
        assert self.action_space.contains(action)
        
        if action == 0:  # Stick
            done = True
            while handSum(self.dealer_hand) < 17:
                self.dealer_hand.append(drawCard(self.np_random))
            reward = compareHands(score(self.player_hand), score(self.dealer_hand))
            if natural(self.player_hand) and reward == 1:
                reward = 1.5 if self.natural else 1
        elif action == 1:  # Hit
            self.player_hand.append(drawCard(self.np_random))
            if bust(self.player_hand):
                done = True
                reward = -1
            else:
                done = False
                reward = 0
        elif action == 2:  # Double down
            
            self.player_hand.append(drawCard(self.np_random))
            if bust(self.player_hand):
                done = True
                reward = -2
            else:
                while handSum(self.dealer_hand) < 17:
                    self.dealer_hand.append(drawCard(self.np_random))
                temp_reward = compareHands(score(self.player_hand), score(self.dealer_hand))
                reward = 2 * temp_reward
                done = True
                
        elif action == 3: # Surrender
            assert len(self.player_hand) == 2
            done = True
            reward = -0.5

        self.actions_taken += 1
        return self._get_obs(), reward, done, {}


---
---

# Model 

The alorithm is taken from Sutton and Barto and is attached below.

![Algorithm](./MCCES.png)

## Monte Carlo Control with Exploring Starts

Our model uses Monte Carlo methods for learning an optimal policy for playing Blackjack. Monte Carlo methods involve running many simulations of the game to gather data on the outcomes of different actions in different states. This data is then used to estimate the expected rewards for each action in each state.

### Algorithm: Monte Carlo with Exploring Starts

1. **Initialization**:
   - Initialize the action-value function Q to arbitrary values.
   - Initialize an empty list for returns for each state-action pair.

2. **Policy**:
   - Use an epsilon-greedy policy for action selection, which allows exploration of the state space by occasionally choosing random actions.

3. **Generating Episodes**:
   - Generate episodes (sequences of state-action-reward tuples) using the current policy.
   - Ensure that all state-action pairs are explored by starting the episodes from random state-action pairs.

4. **Updating Q-values**:
   - For each episode, calculate the return (cumulative reward) for each state-action pair.
   - Update the Q-values as the average of these returns.

5. **Improving the Policy**:
   - Use the updated Q-values to improve the policy, making it more likely to choose actions with higher expected rewards.

### Algorithm Steps
1. Initialize Q and returns.
2. Repeat for a large number of episodes:
   - Generate an episode using the current policy.
   - For each state-action pair in the episode:
     - Calculate the return.
     - Update Q.
     - Update the policy to be epsilon-greedy with respect to the new Q-values.


---

### Hyperparameters and Environment Initialization:

In [12]:
env = BlackJackEnv()

epsilon = 0.1
gamma = 1.0
num_episodes = 500000

Q = defaultdict(lambda: np.zeros(env.action_space.n))
Returns = defaultdict(list)

---

### Generating Episodes:

In [13]:
def generate_episode(policy):
    episode = []
    state = env.reset()
    while True:
        valid_actions = [0, 1, 2, 3] if possible(env.player_hand, env.actions_taken) else [0, 1]
        action_probabilities = np.array([policy(state)[a] for a in valid_actions])
        action_probabilities /= action_probabilities.sum()
        action = np.random.choice(valid_actions, p=action_probabilities)
        next_state, reward, done, _ = env.step(action)
        episode.append((state, action, reward))
        state = next_state
        if done:
            break
    return episode

---

### Policy Initialization ($\epsilon$-greedy):


In [14]:
def create_policy(Q, epsilon, num_actions):
    def policy_fn(state):
        valid_actions = [0, 1, 2, 3] if possible(env.player_hand, env.actions_taken) else [0, 1]
        policy = np.zeros(num_actions)
        for action in valid_actions:
            policy[action] = epsilon / len(valid_actions)
        best_action = valid_actions[np.argmax([Q[state][a] for a in valid_actions])]
        policy[best_action] += (1.0 - epsilon)
        return policy
    return policy_fn

policy = create_policy(Q, epsilon, env.action_space.n)
average_returns = []
average_returns_last_10000 = []

---

### Updating Q-values:

In [15]:
for i in range(num_episodes):
    episode = generate_episode(policy)
    G = 0
    visited_state_action_pairs = set()
    for state, action, reward in reversed(episode):
        G = gamma * G + reward
        if (state, action) not in visited_state_action_pairs:
            Returns[(state, action)].append(G)
            Q[state][action] = np.mean(Returns[(state, action)])
            visited_state_action_pairs.add((state, action))
    policy = create_policy(Q, epsilon, env.action_space.n)
    
    if (i + 1) % 10000 == 0:
        average_return = np.mean([sum([reward for (_, _, reward) in generate_episode(policy)]) for _ in range(1000)])
        average_returns.append(average_return)
        if len(average_returns) > 1:
            avg_last_10000 = np.mean(average_returns[-10:])
        else:
            avg_last_10000 = average_return
        average_returns_last_10000.append(avg_last_10000)
        print(f'Episode {i+1}/{num_episodes}: Average Return = {average_return}, Last 10000 Average = {avg_last_10000}')
        
print("Training completed.")

Episode 10000/500000: Average Return = -0.219, Last 10000 Average = -0.219
Episode 20000/500000: Average Return = -0.1885, Last 10000 Average = -0.20375
Episode 30000/500000: Average Return = -0.139, Last 10000 Average = -0.18216666666666667
Episode 40000/500000: Average Return = -0.164, Last 10000 Average = -0.177625
Episode 50000/500000: Average Return = -0.1415, Last 10000 Average = -0.1704
Episode 60000/500000: Average Return = -0.118, Last 10000 Average = -0.16166666666666665
Episode 70000/500000: Average Return = -0.1055, Last 10000 Average = -0.15364285714285714
Episode 80000/500000: Average Return = -0.1225, Last 10000 Average = -0.14975
Episode 90000/500000: Average Return = -0.131, Last 10000 Average = -0.14766666666666667
Episode 100000/500000: Average Return = -0.152, Last 10000 Average = -0.14809999999999998
Episode 110000/500000: Average Return = -0.117, Last 10000 Average = -0.13789999999999997
Episode 120000/500000: Average Return = -0.0835, Last 10000 Average = -0.1274

---

### Final Policy:

In [16]:
state_space = [(sum_hand, dealer_card, usable_ace, double_down) for sum_hand in range(4, 22)
               for dealer_card in range(1, 11) for usable_ace in [False, True] for double_down in [False, True]]

policy_matrix = np.zeros((32, 11, 2, 2), dtype=int)
for (player_sum, dealer_card, usable_ace, double_down) in state_space:
    state = (player_sum, dealer_card, usable_ace, double_down)
    if state in Q:
        best_action = np.argmax(Q[state])
        policy_matrix[player_sum, dealer_card, usable_ace, double_down] = best_action

---

### Evaluating final Policy:

In [17]:
num_test_episodes = 100000
total_rewards = 0
action_counts = np.zeros(4)

for _ in range(num_test_episodes):
    state = env.reset()
    episode_rewards = 0
    while True:
        valid_actions = [0, 1, 2, 3] if possible(env.player_hand, env.actions_taken) else [0, 1]
        action = np.argmax([Q[state][a] for a in valid_actions])
        next_state, reward, done, _ = env.step(action)
        episode_rewards += reward
        action_counts[action] += 1
        state = next_state
        if done:
            break
    total_rewards += episode_rewards

average_return = total_rewards / num_test_episodes
print(f"Final average return over {num_test_episodes} games: {average_return}")
print(f"Number of hits: {action_counts[1]}")
print(f"Number of stands: {action_counts[0]}")
print(f"Number of double downs: {action_counts[2]}")
print(f"Number of surrenders: {action_counts[3]}")

Final average return over 100000 games: -0.046725
Number of hits: 31286.0
Number of stands: 64845.0
Number of double downs: 10826.0
Number of surrenders: 17993.0


---
---

## Conclusion:
Even after such training and palying ```1,00,000``` games, our policy could not return a positive value which probably tells us to stay away from gambling :)

A personal note: I tried a lot of different stuff to add another action ```split``` but that simply wouldn't work and the code was filled with bugs so at the end I finally decided to remove it to save some troubles.

-----